In [21]:
import pandas as pd
import nltk, re, pprint
import string
from nltk import word_tokenize

In [3]:
df = pd.read_csv("mypersonality_final.csv", encoding = "ISO-8859-1")

In [4]:
print(df.describe())
print(df.head(200))
print(len(df))

              sEXT         sNEU         sAGR         sCON         sOPN  \
count  9917.000000  9917.000000  9917.000000  9917.000000  9917.000000   
mean      3.354760     2.609453     3.616643     3.474201     4.130386   
std       0.857578     0.760248     0.682485     0.737215     0.585672   
min       1.330000     1.250000     1.650000     1.450000     2.250000   
25%       2.710000     2.000000     3.140000     3.000000     3.750000   
50%       3.400000     2.600000     3.650000     3.400000     4.250000   
75%       4.000000     3.050000     4.150000     4.000000     4.550000   
max       5.000000     4.750000     5.000000     5.000000     5.000000   

        NETWORKSIZE   BETWEENNESS  NBETWEENNESS       DENSITY     BROKERAGE  \
count   9917.000000  9.917000e+03   9917.000000   9917.000000  9.917000e+03   
mean     429.377120  1.354253e+05     94.665170      3.154012  1.376425e+05   
std      428.760382  1.994338e+05      5.506696    311.073343  2.013921e+05   
min       24.0000

In [5]:
decide per feature whether it's post-based or person-based

SyntaxError: invalid syntax (<ipython-input-5-13b8819e5f08>, line 1)

# meta information

* time stamp -> DATE
* language -> english
* location -> might be interesting, but hard to get
* company location vs follower location
* number of urls in a message
* (number of friends)
* (number of shares)
* (number of likes)

# lexical

## char-based
* number of chars
* number of ascii chars
* number of ascii uppercase chars
* number of ascii lowercase chars
* number of digits
* number of whitespaces
* number of tabs
* number of special chars
* freq of special chars
* freq of letters

## word-based
* number of words
* number of short words
* number of unique words
* number of sentences
* short words / total ratio
* avg word length
* avg sentence length (chars)
* avg sentence length (words)
* avg line length
* word length distribution per word length
* word usage ratio (per 100 words) -> Hapax legomena (freq of once occuring words)

-> choose features that don't correlate
-> play around

* vocabulary richness measures? -> Zheng / Tweedie and Baayen
* number of unique words over one person

In [19]:
### configurations

chosen_punctuation = ['.', ',', ';', ':', '!', '?', '"', "'"]
special_chars = set(string.punctuation) - set(chosen_punctuation)
end_of_sentence = ['.', '?', '!']
print(special_chars)

{'>', '|', '{', '<', '$', '=', '%', ')', '[', '}', ']', '+', '&', '@', '#', '`', '*', '^', '_', '-', '\\', '~', '/', '('}


In [7]:
# number of chars
df['total_chars'] = df['STATUS'].apply(len)


# number of ascii chars
df['total_alpha_chars'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.isalnum()))


# number of ascii uppercase chars
df['total_upper_chars'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.isupper()))


# number of ascii lowercase chars
df['total_lower_chars'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.islower()))


# number of digits
df['total_digits'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.isdigit()))


# number of whitespaces
df['total_whitespaces'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.isspace()))


# number of tabs
df['total_tabs'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c=='\t'))


# number of special chars
df['total_whitespaces'] = df['STATUS'].apply(lambda s: sum(1 for c in s if c.isspace()))


# freq of letters
for letter in string.ascii_lowercase:
    letter_upper = letter.upper()
    df['letter_' + letter] = df['STATUS'].apply(lambda s: sum(1 for c in s if c in [letter, letter_upper]))

    
# freq of special chars
for letter in special_chars:
    df['letter_' + letter] = df['STATUS'].apply(lambda s: sum(1 for c in s if c in [letter]))

print(df.head(20))

                             #AUTHID  \
0   b7b7764cfa1c523e4e93ab2a79a946c4   
1   b7b7764cfa1c523e4e93ab2a79a946c4   
2   b7b7764cfa1c523e4e93ab2a79a946c4   
3   b7b7764cfa1c523e4e93ab2a79a946c4   
4   b7b7764cfa1c523e4e93ab2a79a946c4   
5   b7b7764cfa1c523e4e93ab2a79a946c4   
6   b7b7764cfa1c523e4e93ab2a79a946c4   
7   b7b7764cfa1c523e4e93ab2a79a946c4   
8   b7b7764cfa1c523e4e93ab2a79a946c4   
9   b7b7764cfa1c523e4e93ab2a79a946c4   
10  b7b7764cfa1c523e4e93ab2a79a946c4   
11  b7b7764cfa1c523e4e93ab2a79a946c4   
12  b7b7764cfa1c523e4e93ab2a79a946c4   
13  b7b7764cfa1c523e4e93ab2a79a946c4   
14  b7b7764cfa1c523e4e93ab2a79a946c4   
15  b7b7764cfa1c523e4e93ab2a79a946c4   
16  b7b7764cfa1c523e4e93ab2a79a946c4   
17  b7b7764cfa1c523e4e93ab2a79a946c4   
18  b7b7764cfa1c523e4e93ab2a79a946c4   
19  b7b7764cfa1c523e4e93ab2a79a946c4   

                                               STATUS  sEXT  sNEU  sAGR  sCON  \
0                         likes the sound of thunder.  2.65   3.0  3.15  3.25 

In [59]:
def get_word_based_lexical_features():
    # number of words
    df['tokenized_words'] = df['STATUS'].apply(lambda s: nltk.Text(word_tokenize(s)))
    df['total_words'] = df['tokenized_words'].apply(lambda s: len(s))

    # number of short words (len(word) < 7)
    df['total_short_words'] = df['tokenized_words'].apply(lambda s: sum(1 for word in s if len(word) < 7))

    # number of unique words
    df['total_unique_words'] = df['tokenized_words'].apply(lambda s: len(set(s)))

    # number of sentences
    df['total_sentences'] = df['STATUS'].apply(lambda s: len(re.split('\.\s|\?+\s|\!+\s', s)))

    # short words / total ratio


    # avg word length


    # avg sentence length (chars)


    # avg sentence length (words)


    # avg line length


    # word length distribution per word length


    # word usage ratio (per 100 words) -> Hapax legomena (freq of once occuring words)

get_word_based_lexical_features()
df.head(100)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,...,letter_\,letter_~,letter_/,letter_(,tokenized_words,total_words,total__short_words,total_short_words,total_unique_words,total_sentences
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(likes, the, sound, of, thunder, .)",6,5,5,6,1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(is, so, sleepy, it, 's, not, even, funny, tha...",17,17,17,16,1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(is, sore, and, wants, the, knot, of, muscles,...",31,28,28,28,3
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(likes, how, the, day, sounds, in, this, new, ...",10,10,10,10,1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(is, home, ., <, 3)",5,5,5,5,2
5,b7b7764cfa1c523e4e93ab2a79a946c4,www.thejokerblogs.com,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,(www.thejokerblogs.com),1,0,0,1,1
6,b7b7764cfa1c523e4e93ab2a79a946c4,"saw a nun zombie, and liked it. Also, *PROPNAM...",2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(saw, a, nun, zombie, ,, and, liked, it, ., Al...",23,20,20,20,2
7,b7b7764cfa1c523e4e93ab2a79a946c4,is in Kentucky. 421 miles into her 1100 mile j...,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(is, in, Kentucky, ., 421, miles, into, her, 1...",13,11,11,12,2
8,b7b7764cfa1c523e4e93ab2a79a946c4,was about to finish a digital painting before ...,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(was, about, to, finish, a, digital, painting,...",29,23,23,26,2
9,b7b7764cfa1c523e4e93ab2a79a946c4,is celebrating her new haircut by listening to...,2.65,3.0,3.15,3.25,4.4,n,y,n,...,0,0,0,0,"(is, celebrating, her, new, haircut, by, liste...",17,11,11,17,1


In [62]:
print(df['STATUS'][2])
re.split('\.\s|\?+\s|\!+\s', df['STATUS'][2])

is sore and wants the knot of muscles at the base of her neck to stop hurting. On the other hand, YAY I'M IN ILLINOIS! <3


['is sore and wants the knot of muscles at the base of her neck to stop hurting',
 "On the other hand, YAY I'M IN ILLINOIS",
 '<3']

In [10]:
df['tokenized_words'].unique()

array([<Text: likes the sound of thunder ....>,
       <Text: is so sleepy it 's not even funny...>,
       <Text: is sore and wants the knot of muscles...>, ...,
       <Text: is studying hard for the G.R.E ....>,
       <Text: snipers get more head...>,
       <Text: Last night was amazing ! Not only did...>], dtype=object)

# syntactic

* character freq of punctuation
* freq of function words -> which ones? libraries?
prepositions..


* part of speech (POS) -> might be immature
Hypothesis: differs too much between persons and brands

# structural

* number of lines
* number of blank lines
* has a greeting
* (has a farewell) -> might not exist, might differ between persons and brands


* (paragraph length)
* (number of sentences per paragraph)
* (number of words per paragraph)
* (separators per paragraph)
* (quoted content -> special chars?)
* (indentation)

# content-specific

* emoticons
* abbreviations
* number of hashtags
* where are the hashtags


* freq of keywords -> might be a bit hard
* message similiarity score

# n-grams

* word n-grams
* character n-grams